# Clustering Project Notebook

![Image of Zillow Logo](https://www.roundtablerealty.com/uploads/agent-1/699119.png)




### Zillow: What is driving the errors in the Zestimates?
#### Background:
 Zillow, a real estate website in the United States, uses a Zestimate to estimate a property's market value.
> What is a Zestimate?     & 
How is the Zestimate calculated? - zillow.com

> For this project, we will look into finding drivers of error in the Zestimate. Using clustering methodologies, feature identification and comparison, visualizations, statistical testing, and regression models, we are to find drivers of error to predict log error.

### Project Goals:
 - Using Zillow data for single unit/single family homes sold in 2017, find drivers of log error in the Zestimate.

 - Create 4 regression models to predict log error

 - Use clustering methodologies to help identify drivers of log error

 - Deliver findings in a final Jupyter Notebook presentation

## Executive Summary:
#### Findings Include:
#### Takeaways:
#### Next Steps to Explore Further:

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Statistical Tests
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve
import env

# supress scientific notation
np.set_printoptions(suppress=True)

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
import wrangle

In [2]:
df = wrangle.get_zillow_data()
df = df.dropna()

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'month' in 'field list'")
[SQL:  SELECT bedroomcnt, 
                                     bathroomcnt, 
                                     calculatedfinishedsquarefeet, 
                                     yearbuilt, 
                                     regionidzip, 
                                     fips,
                                     taxvaluedollarcnt,
                                     logerror,
                                     month,
                                     longitude,
                                     latitude
                              FROM properties_2017
                              JOIN predictions_2017 USING (parcelid)
                              WHERE propertylandusetypeid = 261;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [ ]:
df.head()

## Data Acquisition / Prep / Wrangling Takeaways:
 - Created and used functions in wrangle.py to acquire and prep data

 - Used a SQL query to join 7 tables

 - Selected only single family properties filtered by:
   - Properties with propertylandusetypeid = 261
   - Only retrieving latest transaction date, eliminating duplicates sold more than once in same year
   - Only selecting properties with at least 1 bath & bed and 350 sqft area
   - Only properties with a latitude and longitude that is not null
   - Dropping columns that are less than 70% populated, mostly empty columns
   - Adding a 'county' column based on FIPS unique county identifier feature
   - Dropping columns that are not useful features or redundant
   - Filling null values in unitcnt column with 1 since all are single unit properties
   - Replacing nulls with median values for select columns:
     - 7313 for lotsizesquarefeet
     - 6.0 for buildingqualitytypeid -Since this is Southern CA, filling null with 'None' for heatingorsystemdesc because most likely don't have one -Rename columns for easier readability -Removed outliers based on Inter Quartile Rule for properties with outliers in square footage, price, bedrooms, and bathrooms

____

# Explore

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Check to see if nulls were removed
wrangle.nulls_by_col(df)

In [ ]:
wrangle.nulls_by_row(df)

In [ ]:
# Plot the distribution of some numeric variables
fig, axs = plt.subplots(1, 5, figsize=(15, 7))

for ax, col in zip(axs, df.select_dtypes('number')):
    df[col].plot.hist(ax=ax, title=col, ec='black')

In [ ]:
# Before exploring data further, we'll need to split data into train, validate and test, 
# so that we only explore on train. The data was split with 'wrangle.wrangle_zillow()' from wrangle.py module
train, validate, test = wrangle.wrangle_zillow()
train, validate, test = wrangle.Min_Max_Scaler(train, validate, test)
train.shape, validate.shape, test.shape

In [ ]:
# Binning square footage for comparison and visualization purposes
train['sq_ft_bins'] = pd.cut(train.squarefeet, [0, 1000, 2000,3000,4000, 10000])
train.head()

In [ ]:
# Binning transactiondate for comparison and visualization purposes
train['years_old'] = 2017 - df['year_built']

In [ ]:
# A look at logerror:
df.logerror.hist()
plt.xlim(-4, 4);

In [ ]:
df.logerror.max(), df.logerror.min(), df.logerror.mean(), df.logerror.median()

In [ ]:
#Let's look at how variables are correlated
corr =  train.corr(method='pearson')

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Generate a custom diverging colormap
cmap = sns.color_palette("coolwarm", as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

### My first question:
## Does log error differ across square feet?

In [ ]:
# Let's look at the relationship between log error and square feet
train.plot.scatter(
    x="logerror",
    y="squarefeet",
    title='Is there a relationship between logerror and square feet?',
    figsize=(8, 6),
);

In [ ]:
# Distribution of square footage 
df.squarefeet.hist();

In [ ]:
# Distribution of square footage by bins
# Looks like most properties are between 1,000-2,000 square feet

train.sq_ft_bins.value_counts().plot.barh(title='Distribution of sq_footage_bins');

In [ ]:
# Let's look at how log error compares across square footage by bins
# It doesn't look like there's much difference 
sns.violinplot(data=train, y='logerror', x='sq_ft_bins');

## Levene's Test: test for equal variance
$Ho$: There is no difference in logerror variance between the square footage

$Ha$: There is a difference in logerror variance between the square footage

In [ ]:
stats.levene(
    train[train.squarefeet <= 1000].logerror,
    train[train.squarefeet > 1000].logerror,
)

In [ ]:
stats.levene(
    train[train.squarefeet <= 2000].logerror,
    train[train.squarefeet > 2000].logerror,
)

In [ ]:
stats.levene(
    train[train.squarefeet <= 3000].logerror,
    train[train.squarefeet > 3000].logerror,
)

In [ ]:
stats.levene(
    train[train.squarefeet <= 4000].logerror,
    train[train.squarefeet > 4000].logerror,
)

### Create independent statistical tests to check for dependency amongst variables

In [ ]:
stats.ttest_ind(
    train[train.squarefeet <= 1000].logerror,
    train[train.squarefeet > 1000].logerror,
    equal_var=False,
)

In [ ]:
stats.ttest_ind(
    train[train.squarefeet <= 2000].logerror,
    train[train.squarefeet > 2000].logerror,
    equal_var=False,
)

In [ ]:
stats.ttest_ind(
    train[train.squarefeet <= 3000].logerror,
    train[train.squarefeet > 3000].logerror,
    equal_var=False,
)

In [ ]:
stats.ttest_ind(
    train[train.squarefeet <= 4000].logerror,
    train[train.squarefeet > 4000].logerror,
    equal_var=False,
)

### Conclusion:
We reject the null hypothesis that the logerror is the same for all square footages.

-----

### Hypothesis :
$Ho$ : Null Hypothesis: There is no correlation between square footage and log error

$Ha$ : Alternative Hypothesis: Square footage and log error are correlated

In [ ]:
train = train.dropna()
test = test.dropna()
validate = validate.dropna()

In [ ]:
x = train.squarefeet
y = train.logerror

In [ ]:
corr, p = stats.pearsonr(x, y)
corr, p

In [ ]:
# Null hypothesis is rejected.  Square footage and log error are correlated.

### My second question: 
## Does log error differ across county?

In [ ]:
df.head()

In [ ]:
# Look at how counties compare in number
### 6037 = Los Angeles County
### 6059 = Orange County
### 6111 = Ventura County
df.fips.value_counts()

In [ ]:
df['county'] = df.fips

In [ ]:
df.head()

In [ ]:
#Look at how log error compares amongst counties
sns.relplot(data=train, x='logerror', y='fips');

In [ ]:
#Plotting distribution of log error by county
# 6037 (Los Angeles)
# 6059 (Orange)
# 6111 (Ventura)
sns.displot(train, x='logerror', hue='fips', kind='kde', fill=True)
plt.title('Log Error by County')
plt.xlabel('Log Error')
plt.xlim(-0.75, 0.75)
plt.show()

### Levene's Test: test for equal variance
$Ho$: There is no difference in logerror variance between the counties

$Ha$: There is a difference in logerror variance between the counties

In [ ]:
stats.levene(
    train[train.fips == 6037.0].logerror,
    train[train.fips != 6037.0].logerror,
)

In [ ]:
stats.levene(
    train[train.fips == 6059.0].logerror,
    train[train.fips != 6059.0].logerror,
)

In [ ]:
stats.levene(
    train[train.fips == 6111.0].logerror,
    train[train.fips != 6111.0].logerror,
)

In [ ]:
sns.scatterplot(data=train, y='logerror', x='fips');

### Conclusion:
We reject the null hypothesis that the variance in logerror is the same for all counties. The variance is not equal.

### My third question:
## Question 4: Does log error differ for different number of bedrooms?

In [ ]:
ax = train.bedrooms.hist()
plt.title('Histogram: Bedrooms')
plt.xlabel('Bedrooms')
plt.ylabel('Count')
ax.axvline(2, color='black')
ax.axvline(4, color='black');

In [ ]:
train.plot.scatter(
    y="logerror",
    x="bedrooms",
    title='Is there a relationship between log error and bedrooms?',
    figsize=(8, 6),
);

In [ ]:
sns.violinplot(data=train, y='logerror', x='bedrooms');

### Levene's Test: test for equal variance
$Ho$: There is no difference in logerror variance for different number of bedrooms

$Ha$: There is a difference in logerror variance different number of bedrooms

In [ ]:
stats.levene(
    train[train.bedrooms <= 4].logerror,
    train[train.bedrooms > 4].logerror,
)

In [ ]:
# We fail to reject the null hypothesis that there is no difference in logerror variance for different number of bedrooms.
# The variance is equal.

### Is the log error the same for properties with 4 or less as those with more than 4 bedrooms?
$Ho$: The logerror is the same for properties with 4 bedrooms or less as with those with more than 4 bedrooms

$Ha$: The logerror is not the same for properties with 4 bedrooms or less as with those with 5 bedrooms

In [ ]:
stats.ttest_ind(
   train[train.bedrooms <= 4].logerror,
    train[train.bedrooms > 4].logerror,
    equal_var=True,
)

### Conclusion:
We reject the null hypothesis that the log error is the same for properties with 4 bedrooms or less compared to those with 5 bedrooms. The log error is not the same for properties with 4 bedrooms or less compared to those with 5 bedrooms.

In [ ]:
sns.relplot(
    y='bedrooms',
    x='logerror',
    data=train,
)
plt.title("Does log error depend on number of bedrooms?");

### Hypothesis:
$Ho$: There is no correlation between number of bedrooms and log error

$Ha$: Number of bedrooms and log error are correlated

In [ ]:
x = train.bedrooms
y = train.logerror

In [ ]:
corr, p = stats.pearsonr(x, y)
corr, p

#### Null hypothesis is rejected.  Number of bedrooms and log error are correlated.

# Clustering
### Use Clustering to Identify Possible Features of Interest
 - Not having any features indicating location near the coast, will look at latitude and longitude features to see if we can create a possible feature of interest for modeling and predicting

## Clustering Model #1:

In [ ]:
# Choose two features to create possible cluster
# define independent variables for k-means

X = train[['latitude', 'longitude']]
scaler = MinMaxScaler().fit(X)
X_scaled_lat_long = pd.DataFrame(scaler.transform(X), columns= X.columns).set_index([X.index.values])
# sklearn implementation of KMeans

#define the thing and create 3 clusters
kmeans = KMeans(n_clusters=3, random_state=321)

# fit the thing
kmeans.fit(X_scaled_lat_long )

# Use (predict using) the thing 
kmeans.predict(X_scaled_lat_long )
train['cluster'] = kmeans.predict(X_scaled_lat_long )

X_scaled_lat_long['cluster'] = kmeans.predict(X_scaled_lat_long )

# look at the dataframe
X_scaled_lat_long.head()